# BEE 4750 Homework 4: Linear Programming and Capacity Expansion

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 11/07/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to formulate and solve a resource allocation
    problem using linear programming.
-   Problem 2 asks you to formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   Problem 3 (5750 only) asks you to add a CO<sub>2</sub> constraint to
    the capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/Julia Scripts/hw4-teamotter`
   Installed ArgCheck ─────── v2.3.0
   Installed DisplayAs ────── v0.1.6
   Installed MarkdownTables ─ v1.1.0
Precompiling project...
  ✓ ArgCheck
  ✓ DisplayAs
  ✓ MarkdownTables
  3 dependencies successfully precompiled in 10 seconds. 188 already precompiled.


In [2]:
using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables

## Problems (Total: 50/60 Points)

### Problem 1 (20 points)

A farmer has access to a pesticide which can be used on corn, soybeans,
and wheat fields, and costs \$70/ha-yr to apply. The crop yields the
farmer can obtain following crop yields by applying varying rates of
pesticides to the field are shown in
<a href="#tbl-yields" class="quarto-xref">Table 1</a>.

| Application Rate (kg/ha) | Soybean (kg/ha) | Wheat (kg/ha) | Corn (kg/ha) |
|:------------------------:|:---------------:|:-------------:|:------------:|
|            0             |      2900       |     3500      |     5900     |
|            1             |      3800       |     4100      |     6700     |
|            2             |      4400       |     4200      |     7900     |

Table 1: Crop yields from applying varying pesticide rates for Problem
1.

The costs of production, *excluding pesticides*, for each crop, and
selling prices, are shown in
<a href="#tbl-costs" class="quarto-xref">Table 2</a>.

|   Crop   | Production Cost (\$/ha-yr) | Selling Price (\$/kg) |
|:--------:|:--------------------------:|:---------------------:|
| Soybeans |            350             |         0.36          |
|  Wheat   |            280             |         0.27          |
|   Corn   |            390             |         0.22          |

Table 2: Costs of crop production, excluding pesticides, and selling
prices for each crop.

Recently, environmental authorities have declared that farms cannot have
an *average* application rate on soybeans, wheat, and corn which exceeds
0.8, 0.7, and 0.6 kg/ha, respectively. The farmer has asked you for
advice on how they should plant crops and apply pesticides to maximize
profits over 130 total ha while remaining in regulatory compliance if
demand for each crop (which is the maximum the market would buy) this
year is 250,000 kg?

**In this problem**:

-   Formulate a linear program for this resource allocation problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations). **Tip: Make sure that all of your constraints are
    linear**.
-   Implement the program in `JuMP.jl` and find the solution. How many
    ha should the farmer dedicate to each crop and with what pesticide
    application rate(s)? How much profit will the farmer expect to make?
-   The farmer has an opportunity to buy an extra 10 ha of land. How
    much extra profit would this land be worth to the farmer? Discuss
    why this value makes sense and whether you would recommend the
    farmer should make the purchase.


In [72]:
farm_Profit = Model(HiGHS.Optimizer)
@variable(farm_Profit, A[1:9] >= 0.0)
@objective(farm_Profit, Max, 694*A[1]+948*A[2]+1094*A[3]+665*A[4]+757*A[5]+714*A[6]+908*A[7]+1014*A[8]+1208*A[9])
@constraint(farm_Profit, areaTotal, A[1]+A[2]+A[3]+A[4]+A[5]+A[6]+A[7]+A[8]+A[9]==130)
@constraint(farm_Profit, reg_Soybean, 0.8*A[1]-0.2*A[2]-1.2*A[3] >= 0)
@constraint(farm_Profit, reg_Wheat, 0.7*A[4]-0.3*A[5]-1.3*A[6] >= 0)
@constraint(farm_Profit, reg_Corn, 0.6*A[7]-0.4*A[8]-1.4*A[9] >= 0)
@constraint(farm_Profit, mass_Soybean, 2.9*A[1]+3.8*A[2]+4.4*A[3] <= 250)
@constraint(farm_Profit, mass_Wheat, 3.5*A[4]+4.1*A[5]+4.2*A[6] <= 250)
@constraint(farm_Profit, mass_Corn, 5.9*A[7]+6.7*A[8]+7.9*A[9] <= 250)
print(farm_Profit)


Max 694 A[1] + 948 A[2] + 1094 A[3] + 665 A[4] + 757 A[5] + 714 A[6] + 908 A[7] + 1014 A[8] + 1208 A[9]
Subject to
 areaTotal : A[1] + A[2] + A[3] + A[4] + A[5] + A[6] + A[7] + A[8] + A[9] = 130
 reg_Soybean : 0.8 A[1] - 0.2 A[2] - 1.2 A[3] ≥ 0
 reg_Wheat : 0.7 A[4] - 0.3 A[5] - 1.3 A[6] ≥ 0
 reg_Corn : 0.6 A[7] - 0.4 A[8] - 1.4 A[9] ≥ 0
 mass_Soybean : 2.9 A[1] + 3.8 A[2] + 4.4 A[3] ≤ 250
 mass_Wheat : 3.5 A[4] + 4.1 A[5] + 4.2 A[6] ≤ 250
 mass_Corn : 5.9 A[7] + 6.7 A[8] + 7.9 A[9] ≤ 250
 A[1] ≥ 0
 A[2] ≥ 0
 A[3] ≥ 0
 A[4] ≥ 0
 A[5] ≥ 0
 A[6] ≥ 0
 A[7] ≥ 0
 A[8] ≥ 0
 A[9] ≥ 0


In [73]:
optimize!(farm_Profit)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 8e+00]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
7 rows, 9 cols, 27 nonzeros  0s
6 rows, 8 cols, 28 nonzeros  0s
Presolve : Reductions: rows 6(-1); columns 8(-1); elements 28(+1)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.2109990367e+03 Ph1: 6(27.1); Du: 7(1211) 0s
          5    -1.1674116702e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.1674116702e+05
HiGHS run time      :          0.00


In [74]:
@show value.(A)

value.(A) = [13.812154696132596, 55.248618784530386, 0.0, 6.743306417339568, 15.73438164045899, 0.0, 26.923076923076923, 0.0, 11.538461538461535]


9-element Vector{Float64}:
 13.812154696132596
 55.248618784530386
  0.0
  6.743306417339568
 15.73438164045899
  0.0
 26.923076923076923
  0.0
 11.538461538461535

In [100]:
@expression(farm_Profit, area_Soybean, A[1]+A[2]+A[3])
@show value.(area_Soybean);
@expression(farm_Profit, area_Wheat, A[4]+A[5]+A[6])
@show value.(area_Wheat);
@expression(farm_Profit, area_Corn, A[7]+A[8]+A[9])
@show value.(area_Corn);

@show objective_value(farm_Profit);

value.(area_Soybean) = 69.06077348066299
value.(area_Wheat) = 22.47768805779856
value.(area_Corn) = 38.46153846153846
objective_value(farm_Profit) = 116741.16702082448


In [101]:
farm_Profit2 = Model(HiGHS.Optimizer)
@variable(farm_Profit2, A[1:9] >= 0.0)
@objective(farm_Profit2, Max, 694*A[1]+948*A[2]+1094*A[3]+665*A[4]+757*A[5]+714*A[6]+908*A[7]+1014*A[8]+1208*A[9])
@constraint(farm_Profit2, areaTotal, A[1]+A[2]+A[3]+A[4]+A[5]+A[6]+A[7]+A[8]+A[9]==140)
@constraint(farm_Profit2, reg_Soybean, 0.8*A[1]-0.2*A[2]-1.2*A[3] >= 0)
@constraint(farm_Profit2, reg_Wheat, 0.7*A[4]-0.3*A[5]-1.3*A[6] >= 0)
@constraint(farm_Profit2, reg_Corn, 0.6*A[7]-0.4*A[8]-1.4*A[9] >= 0)
@constraint(farm_Profit2, mass_Soybean, 2.9*A[1]+3.8*A[2]+4.4*A[3] <= 250)
@constraint(farm_Profit2, mass_Wheat, 3.5*A[4]+4.1*A[5]+4.2*A[6] <= 250)
@constraint(farm_Profit2, mass_Corn, 5.9*A[7]+6.7*A[8]+7.9*A[9] <= 250)
optimize!(farm_Profit2)
@show objective_value(farm_Profit2);

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 8e+00]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
7 rows, 9 cols, 27 nonzeros  0s
6 rows, 8 cols, 28 nonzeros  0s
Presolve : Reductions: rows 6(-1); columns 8(-1); elements 28(+1)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.2109990367e+03 Ph1: 6(27.1); Du: 7(1211) 0s
          5    -1.2403516702e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.2403516702e+05
HiGHS run time      :          0.00
objective_value(farm_Profit2) = 124035.16702082448


### Problem 2 (30 points)

For this problem, we will use hourly load (demand) data from 2013 in New
York’s Zone C (which includes Ithaca). The load data is loaded and
plotted below in <a href="#fig-demand" class="quarto-xref">Figure 1</a>.

In [102]:
# load the data, pull Zone C, and reformat the DataFrame
NY_demand = DataFrame(CSV.File("data/2013_hourly_load_NY.csv"))
rename!(NY_demand, :"Time Stamp" => :Date)
demand = NY_demand[:, [:Date, :C]]
rename!(demand, :C => :Demand)
demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(demand.Hour, demand.Demand, xlabel="Hour of Year", ylabel="Demand (MWh)", label=:false)

GKS: dlopen(/Users/rharwell/.julia/artifacts/4cab0d575711b60d292e4bd3088c55d27752a8ea/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/rharwell/.julia/artifacts/4cab0d575711b60d292e4bd3088c55d27752a8ea/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


Next, we load the generator data, shown in
<a href="#tbl-generators" class="quarto-xref">Table 3</a>. This data
includes fixed costs (\$/MW installed), variable costs (\$/MWh
generated), and CO<sub>2</sub> emissions intensity (tCO<sub>2</sub>/MWh
generated).

In [103]:
gens = DataFrame(CSV.File("data/generators.csv"))

Row,Plant,FixedCost,VarCost,Emissions
,String15,Int64,Int64,Float64
1,Geothermal,450000,0,0.0
2,Coal,220000,24,1.0
3,NG CCGT,82000,30,0.43
4,NG CT,65000,40,0.55
5,Wind,91000,0,0.0
6,Solar,70000,0,0.0


Finally, we load the hourly solar and wind capacity factors, which are
plotted in <a href="#fig-cf" class="quarto-xref">Figure 2</a>. These
tell us the fraction of installed capacity which is expected to be
available in a given hour for generation (typically based on the average
meteorology).

In [105]:
# load capacify factors into a DataFrame
cap_factor = DataFrame(CSV.File("data/wind_solar_capacity_factors.csv"))

# plot January capacity factors
p1 = plot(cap_factor.Wind[1:(24*31)], label="Wind")
plot!(cap_factor.Solar[1:(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")
title!("January Capacity Factors")

p2 = plot(cap_factor.Wind[4344:4344+(24*31)], label="Wind")
plot!(cap_factor.Solar[4344:4344+(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")
title!("July Capacity Factors")

display(p1)
display(p2)

You have been asked to develop a generating capacity expansion plan for
the utility in Riley County, NY, which currently has no existing
electrical generation infrastructure. The utility can build any of the
following plant types: geothermal, coal, natural gas combined cycle gas
turbine (CCGT), natural gas combustion turbine (CT), solar, and wind.

While coal, CCGT, and CT plants can generate at their full installed
capacity, geothermal plants operate at maximum 85% capacity, and solar
and wind available capacities vary by the hour depend on the expected
meteorology. The utility will also penalize any non-served demand at a
rate of \$10,000/MWh.

**In this problem**:

-   Formulate a linear program for this capacity expansion problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations).
-   Implement your linear program in `JuMP.jl`. Find the optimal
    solution. How much should the utility build of each type of
    generating plant? What will the total cost be? How much energy will
    be non-served?
-   What fraction of annual generation does each plant type produce? How
    does this compare to the breakdown of built capacity that you found
    in Problem 1.5? Do these results make sense given the generator
    data?
-   Make a plot of the electricity price in each hour. Discuss any
    trends that you see.

> **Significant Digits**
>
> Use `round(x; digits=n)` to report values to the appropriate
> precision! If your number is on a different order of magnitude and you
> want to round to a certain number of significant digits, you can use
> `round(x; sigdigits=n)`.

> **Getting Variable Output Values**
>
> `value.(x)` will report the values of a `JuMP` variable `x`, but it
> will return a special container which holds other information about
> `x` that is useful for `JuMP`. This means that you can’t use this
> output directly for further calculations. To just extract the values,
> use `value.(x).data`.

> **Suppressing Model Command Output**
>
> The output of specifying model components (variable or constraints)
> can be quite large for this problem because of the number of time
> periods. If you end a cell with an `@variable` or `@constraint`
> command, I *highly* recommend suppressing output by adding a
> semi-colon after the last command, or you might find that your
> notebook crashes.

In [157]:
G = 1:nrow(gens[1:end-2, :])
T = 1:nrow(demand)
NSECost = 10000

generator_model = Model(HiGHS.Optimizer)
@variables(generator_model, begin
    X[g in G] >= 0
    Y[g in G, t in T] >= 0
    NSE[t in T] >= 0
    end)
@objective(generator_model, Min,
    sum(gens[G, :FixedCost] .* X) + sum(gens[G, :VarCost] .* sum(Y[:, t] for t in T)) + NSECost * sum(NSE)
    )
@constraint(generator_model, load_hourly[t in T], sum(Y[:, t]) + NSE[t] >= demand.Demand[t])
@constraint(generator_model, geothermal_capacity[t in T], Y[1, t in T] <= 0.85 * X[1] )
@constraint(generator_model, coal_capacity[t in T], Y[2, t in T] <= X[2])
@constraint(generator_model, combined_capacity[t in T], Y[3, t in T] <= X[3])
@constraint(generator_model, combustion_capacity[t in T], Y[4, t in T] <= X[4])
#@constraint(generator_model, wind_capacity[t in T], Y[5, t in T] <= cap_factor.Wind[1:8760] * X[5])
#@constraint(generator_model, solar_capacity[t in T], Y[6, t in T] <= cap.factor.Solar[t in T] * X[6])
@constraint(generator_model, availability_hourly[g in G, t in T], Y[g, t] <= X[g])
print(generator_model)

Min 450000 X[1] + 220000 X[2] + 82000 X[3] + 65000 X[4] + 24 Y[2,1] + 24 Y[2,2] + 24 Y[2,3] + 24 Y[2,4] + 24 Y[2,5] + 24 Y[2,6] + 24 Y[2,7] + 24 Y[2,8] + 24 Y[2,9] + 24 Y[2,10] + 24 Y[2,11] + 24 Y[2,12] + 24 Y[2,13] + 24 Y[2,14] + 24 Y[2,15] + 24 Y[2,16] + 24 Y[2,17] + 24 Y[2,18] + 24 Y[2,19] + 24 Y[2,20] + 24 Y[2,21] + 24 Y[2,22] + 24 Y[2,23] + 24 Y[2,24] + 24 Y[2,25] + 24 Y[2,26] + [[...34984 terms omitted...]] + 10000 NSE[8731] + 10000 NSE[8732] + 10000 NSE[8733] + 10000 NSE[8734] + 10000 NSE[8735] + 10000 NSE[8736] + 10000 NSE[8737] + 10000 NSE[8738] + 10000 NSE[8739] + 10000 NSE[8740] + 10000 NSE[8741] + 10000 NSE[8742] + 10000 NSE[8743] + 10000 NSE[8744] + 10000 NSE[8745] + 10000 NSE[8746] + 10000 NSE[8747] + 10000 NSE[8748] + 10000 NSE[8749] + 10000 NSE[8750] + 10000 NSE[8751] + 10000 NSE[8752] + 10000 NSE[8753] + 10000 NSE[8754] + 10000 NSE[8755] + 10000 NSE[8756] + 10000 NSE[8757] + 10000 NSE[8758] + 10000 NSE[8759] + 10000 NSE[8760]
Subject to
 load_hourly[1] : Y[1,1] + Y[2,1

In [158]:
optimize!(generator_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-01, 1e+00]
  Cost   [2e+01, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 3e+03]
Presolving model
78840 rows, 43804 cols, 183960 nonzeros  0s
43800 rows, 43803 cols, 113880 nonzeros  0s
43799 rows, 43802 cols, 113878 nonzeros  0s
Presolve : Reductions: rows 43799(-35041); columns 43802(-2); elements 113878(-70082)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(1.63679e+07) 0s
      27989     7.2267190900e+08 Pr: 0(0) 1s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 27989
Objective value     :  7.2267190900e+08
HiGHS run time      :          0.73


In [159]:
@show value.(X);
@show objective_value(generator_model);
@expression(generator_model, total_NSE, sum(NSE));
@show value.(total_NSE)

value.(X) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{Float64}:
   -0.0
    0.0
 2131.8
  794.5999999999999
objective_value(generator_model) = 7.22671909e8
value.(total_NSE) = 216.69999999999982


216.69999999999982

In [145]:
#@expression(generator_model, fraction_geo, sum(Y[1,T])/sum(demand.Demand))
#@show value.(fraction_geo);
#@expression(generator_model, fraction_coal, sum(Y[2,T]))
#@show value.(fraction_coal);
#@expression(generator_model, fraction_combinedcycle, sum(Y[3,T]))
#@show value.(fraction_combinedcycle);
@expression(generator_model, fraction_combustionturb, sum(Y[4,T])/sum(demand.Demand))
@show value.(fraction_combustionturb);
#@expression(generator_model, fraction_wind, sum(Y[5,T]))
#@show value.(fraction_wind);
#@expression(generator_model, fraction_solar, sum(Y[5,T]))
#@show value.(fraction_solar);

value.(fraction_combustionturb) = 0.0011672659970509722


0.0011672659970509722

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

The NY state legislature is considering enacting an annual
CO<sub>2</sub> limit, which for the utility would limit the emissions in
its footprint to 1.5 MtCO<sub>2</sub>/yr.

**In this problem**:

-   Reformulate your linear program from Problem 2 with any necessary
    changes to capture the CO<sub>2</sub> limit.
-   Implement the new optimization problem and find the optimal
    solution. How much should the utility build of each type of
    generating plant? What is different from your plan from Problem 1?
    Do these changes make sense?
-   What would the value to the utility be of allowing it to emit an
    additional 1000 tCO<sub>2</sub>/yr? An additional 5000?

## References

List any external references consulted, including classmates.